# LocalLab Server Test Notebook

This notebook helps test the LocalLab server package on Google Colab.

In [ ]:
# First, let's make sure we have a clean environment
!pip uninstall -y locallab
!pip cache purge

In [ ]:
# Install required dependencies first
!pip install --upgrade transformers accelerate
!pip install torch pyngrok fastapi uvicorn huggingface_hub

In [ ]:
# Install the package from TestPyPI
!pip install locallab

In [ ]:
# Set environment variables
import os
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Set your ngrok token
NGROK_TOKEN = input("Enter your ngrok token: ").strip()
if not NGROK_TOKEN:
    raise ValueError("Ngrok token is required for running the server")
os.environ["NGROK_AUTH_TOKEN"] = NGROK_TOKEN

# Choose model configuration
print("\nAvailable default models:")
print("1. microsoft/phi-2 (Default, 2.7B parameters)")
print("2. TinyLlama/TinyLlama-1.1B-Chat-v1.0 (1.1B parameters)")
print("3. stabilityai/stable-code-3b (3B parameters)")
print("4. Custom model from Hugging Face")

choice = input("\nChoose model (1-4, default is 1): ").strip() or "1"

# Configure model using CLI
if choice == "4":
    custom_model = input("Enter Hugging Face model ID (e.g., meta-llama/Llama-3.2-3B-Instruct): ").strip()
    if not custom_model:
        raise ValueError("Custom model ID is required when choosing option 4")
    !locallab config --model {custom_model}
else:
    models = {
        "1": "microsoft/phi-2",
        "2": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        "3": "stabilityai/stable-code-3b"
    }
    model_name = models.get(choice, "microsoft/phi-2")
    !locallab config --model {model_name}

# Configure performance settings using CLI
!locallab config --flash-attention true --attention-slicing true --cpu-offload true \
                 --better-transformer true --quantize true --quantize-type int8 \
                 --min-free-memory 2000

logging.info(f"Configuration complete")

In [ ]:
# Import and check if locallab is installed correctly
try:
    import locallab
    print("Successfully imported locallab")
except ImportError as e:
    print(f"Import error: {e}")
    print("\nTrying to find the module:")
    !find /usr/local/lib/python3.* -name "locallab*"
    raise

In [ ]:
# Start the server using CLI (notice the ! prefix)
try:
    # Start with the configured model and settings
    !locallab start --use-ngrok
except Exception as e:
    print(f"Error starting server: {str(e)}")
    print("\nTrying to fall back to default model...")
    try:
        # Try with a smaller model and optimizations
        !locallab config --model phi-2 --quantize true --quantize-type int8
        !locallab start --use-ngrok
    except Exception as e2:
        print(f"Fallback also failed: {str(e2)}")
        print("\nPlease try the following:")
        print("1. Restart the runtime")
        print("2. Check your internet connection")
        print("3. Verify your ngrok token")
        print("4. Try a smaller model")